# 실습 내용
- https://www.notion.so/231213-Selenium-a3e6d11827594c8a959c105e17073687

## 원티드에 접속하여 서울에 있는 PM 채용 공고를 스크래핑하기
- 1. 원티드 PM 채용 공고 홈페이지에 접속한다 (https://www.wanted.co.kr/wdlist/507/559?country=kr&job_sort=job.latest_order&years=-1&locations=seoul.all)
- 2. 화면에 보이는 채용 공고 정보를 받아온다.
- 3. 채용 공고를 하나하나 접속해서 채용 공고에 있는 회사, 직무 정보를 추출한다. (advanced 과제: 자격 요건, 위치 정보까지 찾아 저장하기 + 이미지도 따로 찾아 저장하기)
- 4. 엑셀 파일로 저장한다.

In [3]:
# 모듈을 가져오고 브라우저 옵션 정의하기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, os, pandas as pd, urllib.request

options = Options()
options.add_argument('--window-size=1800,1047')
options.add_argument('--window-position=0,0')
options.add_experimental_option("detach", True)

# 폴더 이름 정하고 폴더 생성하기
now = time.localtime()
date_format = '%04d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday)
f_dir = f'{os.getcwd()}\\채용공고_{date_format}'
os.makedirs(f_dir)

# 브라우저 실행하기
URL = 'https://www.wanted.co.kr/wdlist/507/559?country=kr&job_sort=job.latest_order&years=-1&locations=seoul.all'
driver = webdriver.Chrome(executable_path="C:/Users/Jiyoung/Documents/workspace/chromedriver.exe")
#driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)

jobDescription = {
    "회사":[],
    "직무":[]
}

#채용 정보를 받아 저장하는 함수 만들기
def getJobDescription() :
    driver.execute_script("window.scrollTo(0, 400)")
    for i in range(1,12): # 채용 공고의 수는 임의로 12개로 제한 (공고 중간에 삽입되어 있는 광고 때문에...)
        jd = driver.find_element(By.XPATH,f"/html/body/div[1]/div[3]/div/div/div[4]/ul/li[{i}]/div/a/div/div[1]")
        jd.click()
        time.sleep(2)
        
        html = driver.page_source
        html_dom = BeautifulSoup(html, 'lxml')
        
        title = html_dom.find('h2').text
        company = driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/div[1]/div[1]/div[1]/section[2]/div[1]/h6/a").text

        jobDescription["회사"].append(company)
        jobDescription["직무"].append(title)

        driver.back()
        time.sleep(2)
        if i%4 == 0:
            driver.execute_script("window.scrollTo(0, 900)") # 원티드 구조 : 상세페이지에서 뒤로가기 할 때마다 자동으로 '새로고침' 기능 작동
    print("----------데이터 추출 완료----------")

#채용 정보를 파일로 저장하는 함수 만들기
def exportToExcel():
    df = pd.DataFrame(jobDescription)
    filename = f'채용공고_{date_format}.xlsx'
    df.to_excel(f_dir+'\\'+filename)
    print("----------파일 저장 완료----------")

#함수 실행하기
getJobDescription()
exportToExcel()

C:\Users\Jiyoung\AppData\Local\Temp\ipykernel_19656\1074423603.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Jiyoung/Documents/workspace/chromedriver.exe")


----------데이터 추출 완료----------
----------파일 저장 완료----------


In [5]:
driver.quit()

## 풀지 못한 문제점
- 페이지 최하단에 있는 위치 정보를 가져오기 실패
- 채용정보 사이에 삽입되어 있는 '광고창'을 뛰어넘어서 수집을 이어갈 수 없었음 
    - 공고 4개씩 * 3행 + 광고배너(가로길이 full) + 공고 4개씩 * 3행 ... 형식
- 스크래핑을 못하게 막아놓은 느낌이 들었음
- BeautifulSoup을 이용해서 찾아도, Selenium을 이용해서 찾아도 계속 문제가 발생
    - BeatifulSoup = CSS selector 값을 찾아 find나 select 메서드로 데이터를 받아오면 빈데이터가 받아짐
    - Selenium = 요소의 위치 자체를 못 찾음. 위치를 못 찾으니 execute_script(”arguments[0].scrollIntoView()”)로 스크롤을 내리는 것도 안 됨